# Netflix Titles Analysis
Initial workflow outline for exploring the Netflix titles dataset.

## 1. Setup
- Import libraries
- Load `data/netflix_titles.csv`
- Inspect schema

In [ ]:
import pandas as pd

# Load dataset
netflix_df = pd.read_csv('../data/netflix_titles.csv')
netflix_df.head()


## 2. Data Cleaning
- Handle missing values
- Standardize date fields
- Normalize categorical values (e.g., country, listed_in)

In [ ]:
# Placeholder for cleaning steps
# Example: netflix_df['date_added'] = pd.to_datetime(netflix_df['date_added'])


## 3. Exploratory Analysis
- Content distribution by type (Movie vs TV Show)
- Top countries and genres
- Release year trends

In [ ]:
# Placeholder for EDA visualizations
# Example: netflix_df['type'].value_counts()


## 4. Insights for Dashboard
Document metrics and visuals to build in Tableau.

In [ ]:
# Collect key aggregates for Tableau export
# Example: netflix_df.to_csv('../data/netflix_clean.csv', index=False)
